## Name: Sai Surya Salalith Mantha
## USC_ID: 9463553709
## Email_ID: Saisurym@usc.edu

In [1]:
import tensorflow as tf
import pickle
import random
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
np.set_printoptions(threshold=np.nan)

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
model_path = './image_classification'
batch_size = 64
n_samples = 10
top_n_predictions = 5


def unpickle(file):

    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

train=unpickle("cifar-100-python/train")
test=unpickle("cifar-100-python/test")
meta=unpickle("cifar-100-python/meta")

Xtest=test[b'data'].reshape((len(test[b'data']), 3, 32, 32)).transpose(0, 2, 3, 1)
ytest=np.array(test[b'fine_labels'])

Xtest=Xtest.astype(np.float32)


sub=np.load("mean_vec.npy")
Xtest=Xtest-sub

ytestC=np.array(test[b'coarse_labels'])

lab={}
for i in range(0,20):
    lab[i]=set()
for i in range(0,len(ytest)):
    lab[ytestC[i]].add(ytest[i])
    

name_100_labels=[str(i)[2:-1] for i in meta[b'fine_label_names']]
name_20_labels=[str(i)[2:-1] for i in meta[b'coarse_label_names']]


In [3]:
def batch_features_labels(features, labels, batch_size):
    for start in range(0, len(features), batch_size):
        end = min(start + batch_size, len(features))
        yield features[start:end], labels[start:end]

In [4]:
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:

    loader = tf.train.import_meta_graph(model_path + '.meta')
    loader.restore(sess, model_path)
    loaded_x = loaded_graph.get_tensor_by_name('input_x:0')
    loaded_y = loaded_graph.get_tensor_by_name('output_y:0')
    loaded_logits = loaded_graph.get_tensor_by_name('logits:0')
    loaded_acc = loaded_graph.get_tensor_by_name('accuracy:0')

    label_predictor = sess.run(
            tf.nn.top_k(tf.nn.softmax(loaded_logits),1),
            feed_dict={loaded_x: Xtest, loaded_y: ytest})
#         print(label_predictor[1])

    matFine=confusion_matrix(ytest,label_predictor[1])
    accFine=accuracy_score(ytest,label_predictor[1])

    print("---------------------------100 classes------------------")
    
    
    
    
    print("\nAccuracy and confusion matrix for 100 labels")
    print("----------------------------------Accuracy=",100*accFine,"------------------------")
    print("\nConfusion-Matrix:\n",matFine)


    
    
#     print("\nPredicted_labels:\n",label_predictor[1].reshape(len(label_predictor[1])).tolist())
#     print("\n Correct_labels:\n",ytest)
    print("\n\n\n\n\n")
    


    coarse_pred=[]
    pred=np.reshape(label_predictor[1],(len(label_predictor[1]))).tolist()
    for i in range(0,len(pred)):
        for pos in lab.keys():
            if(pred[i] in lab[pos]):
                coarse_pred.append(pos)
    matCoa = confusion_matrix(ytestC,coarse_pred)
    accCoa = accuracy_score(ytestC, coarse_pred)
    print("---------------------------20  classes------------------")
    print("Accuracy and confusion matrix for 20 labels")
    print("----------------------------------Accuracy=",100*accCoa,"------------------------")
    print("Confusion-Matrix:\n",matCoa)
    
    

#     random_test_features, random_test_labels = tuple(zip(*random.sample(list(zip(Xtest, ytest)), n_samples)))
    
    
    
    
    Top_3_pred= sess.run(
        tf.nn.top_k(tf.nn.softmax(loaded_logits), 1),
        feed_dict={loaded_x: Xtest, loaded_y: ytest})
    print("---------------------------------------Considering Top-3----------------------------\n")
#     print("Top 5 Classes:\n"Top_5_pred[1])
    acc_top_3=0
    for i in range(0,len(Top_3_pred[1])):
        if(ytest[i] in Top_3_pred[1][i]):
            acc_top_3+=1
    print("------------------------Accuracy for 100 classes= ",100*acc_top_3/len(ytest))
    
    cacc_top_3=0
    for i in range(0,len(ytest)):
        temp=[]
        for j in lab.keys():
            if(ytest[i] in lab[j]):
                temp.append(lab[j])
                break
#         print(temp)
#         print(Top_5_pred[1][i])
        if(len(temp)!=0):
            temp2=set(temp[0]).intersection(Top_3_pred[1][i])
#             print(temp2)
            if(len(temp2)!=0):
                cacc_top_3+=1
    print("------------------------Accuracy for 20 classes= ",100*cacc_top_3/len(ytest))

    
    Top_5_pred= sess.run(
        tf.nn.top_k(tf.nn.softmax(loaded_logits), top_n_predictions),
        feed_dict={loaded_x: Xtest, loaded_y: ytest})
    print("---------------------------------------Considering Top-5----------------------------\n")
#     print("Top 5 Classes:\n"Top_5_pred[1])
    acc_top_5=0
    for i in range(0,len(Top_5_pred[1])):
        if(ytest[i] in Top_5_pred[1][i]):
            acc_top_5+=1
    print("------------------------Accuracy for 100 classes= ",100*acc_top_5/len(ytest))
    
    cacc_top_5=0
    for i in range(0,len(ytest)):
        temp=[]
        for j in lab.keys():
            if(ytest[i] in lab[j]):
                temp.append(list(lab[j]))
                break
        if(len(temp)!=0):
            temp2=set(temp[0]).intersection(Top_5_pred[1][i])
            if(len(temp2)!=0):
                cacc_top_5+=1
    print("------------------------Accuracy for 20 classes= ",100*cacc_top_5/len(ytest))


INFO:tensorflow:Restoring parameters from ./image_classification
---------------------------100 classes------------------

Accuracy and confusion matrix for 100 labels
----------------------------------Accuracy= 39.08 ------------------------

Confusion-Matrix:
 [[57  2  0  0  0  0  0  0  0  1  3  0  0  0  2  1  0  0  0  0  0  0  0  0
   0  0  2  0  0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0  0  0  0  0
   0  0  0  0  0  3  0  0  0  7  0  0  0  0  0  0  0  0  0  0  0  0  5  0
   0  0  0  0  0  0  2  0  0  0  0  9  0  0  0  0  0  0  0  0  4  0  0  0
   0  0  0  0]
 [ 0 48  1  0  0  0  0  1  2  0  1  0  0  0  1  0  1  0  1  0  0  0  0  0
   0  0  3  0  0  0  0  0  3  1  2  0  0  0  0  0  0  1  0  0  3  0  1  0
   1  0  0  2  0  1  3  0  0  0  0  0  0  0  0  0  0  0  0  2  0  1  1  0
   0  1  0  0  0  0  3  0  1  1  1  1  0  0  2  0  0  0  0  2  3  4  0  0
   0  0  0  0]
 [ 1  1 20  1  0  0  1  0  2  0  0  7  0  0  0  0  1  0  0  0  0  2  0  0
   0  0  5  0  1  0  0  0  2  0  1 15  3 

---------------------------------------Considering Top-3----------------------------

------------------------Accuracy for 100 classes=  39.08
------------------------Accuracy for 20 classes=  51.69
---------------------------------------Considering Top-5----------------------------

------------------------Accuracy for 100 classes=  67.66
------------------------Accuracy for 20 classes=  80.71
